In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import json
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)

In [2]:
data=pd.read_excel(r'CleanedData_updated.xlsx')

In [3]:
#https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/

In [4]:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                            'NC':'Non-Consumable',
                                                           'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [5]:
data.loc[data['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Not Applicable"
data['Item_Fat_Content'].value_counts()

Low Fat           6499
Regular           5019
Not Applicable    2686
Name: Item_Fat_Content, dtype: int64

In [6]:
data['Years_Established'] = data['Outlet_Establishment_Year'].apply(lambda x: 2021 - x) 
data= data.drop(columns=['Outlet_Establishment_Year', 'Item_Identifier', 'Outlet_Identifier'])
data.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Item_Type_Combined,Years_Established
0,9.30,Low Fat,0.016047,Dairy,249.8092,Medium,Tier 1,Supermarket Type1,3735.1380,train,Food,22
1,5.92,Regular,0.019278,Soft Drinks,48.2692,Medium,Tier 3,Supermarket Type2,443.4228,train,Drinks,12
2,17.50,Low Fat,0.016760,Meat,141.6180,Medium,Tier 1,Supermarket Type1,2097.2700,train,Food,22
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,Small,Tier 3,Grocery Store,732.3800,train,Food,23
4,8.93,Not Applicable,0.000000,Household,53.8614,High,Tier 3,Supermarket Type1,994.7052,train,Non-Consumable,34


In [41]:
data.to_csv('14sept.csv', index=False)

In [22]:
# create object of Ordinalencoding
encoder_1= ce.OrdinalEncoder(cols=['Outlet_Size'],return_df=True,
                           mapping=[{'col':'Outlet_Size',
'mapping':{'Small':0,'Medium':1,'High':2}}])


encoder_2= ce.OrdinalEncoder(cols=['Outlet_Location_Type'],return_df=True,
                           mapping=[{'col':'Outlet_Location_Type',
'mapping':{'Tier 3':0,'Tier 2':1,'Tier 1':2}}])


In [23]:
#fit and transform train data 
data_transformed = encoder_1.fit_transform(data)

C:\ProgramData\Anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [24]:
data_transformed = encoder_2.fit_transform(data_transformed)

In [25]:
data_transformed.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Item_Type_Combined,Years_Established
0,9.30,Low Fat,0.016047,Dairy,249.8092,1,2,Supermarket Type1,3735.1380,train,Food,22
1,5.92,Regular,0.019278,Soft Drinks,48.2692,1,0,Supermarket Type2,443.4228,train,Drinks,12
2,17.50,Low Fat,0.016760,Meat,141.6180,1,2,Supermarket Type1,2097.2700,train,Food,22
3,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,0,0,Grocery Store,732.3800,train,Food,23
4,8.93,Not Applicable,0.000000,Household,53.8614,2,0,Supermarket Type1,994.7052,train,Non-Consumable,34


In [26]:
data_transformed.columns

Index(['Item_Weight', 'Item_Fat_Content', 'Item_Visibility', 'Item_Type',
       'Item_MRP', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type',
       'Item_Outlet_Sales', 'source', 'Item_Type_Combined',
       'Years_Established'],
      dtype='object')

In [27]:
#encode the data
data_encoded=pd.get_dummies(data_transformed, columns = ['Item_Fat_Content','Outlet_Type','Item_Type_Combined', 'Item_Type'], drop_first=True)
data_encoded.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,source,Years_Established,Item_Fat_Content_Not Applicable,Item_Fat_Content_Regular,...,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods
0,9.30,0.016047,249.8092,1,2,3735.1380,train,22,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5.92,0.019278,48.2692,1,0,443.4228,train,12,0,1,...,0,0,0,0,0,0,0,0,1,0
2,17.50,0.016760,141.6180,1,2,2097.2700,train,22,0,0,...,0,0,0,0,1,0,0,0,0,0
3,19.20,0.000000,182.0950,0,0,732.3800,train,23,0,1,...,1,0,0,0,0,0,0,0,0,0
4,8.93,0.000000,53.8614,2,0,994.7052,train,34,1,0,...,0,0,0,1,0,0,0,0,0,0


In [30]:
#Create object for hash encoder
#encoder=ce.HashingEncoder(cols='Item_Type',n_components=6)
#Fit and Transform Data
#data=encoder.fit_transform(data_encoded)
data=data_encoded

In [31]:
data.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Size,Outlet_Location_Type,Item_Outlet_Sales,source,Years_Established,Item_Fat_Content_Not Applicable,Item_Fat_Content_Regular,...,Item_Type_Fruits and Vegetables,Item_Type_Hard Drinks,Item_Type_Health and Hygiene,Item_Type_Household,Item_Type_Meat,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods
0,9.30,0.016047,249.8092,1,2,3735.1380,train,22,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5.92,0.019278,48.2692,1,0,443.4228,train,12,0,1,...,0,0,0,0,0,0,0,0,1,0
2,17.50,0.016760,141.6180,1,2,2097.2700,train,22,0,0,...,0,0,0,0,1,0,0,0,0,0
3,19.20,0.000000,182.0950,0,0,732.3800,train,23,0,1,...,1,0,0,0,0,0,0,0,0,0
4,8.93,0.000000,53.8614,2,0,994.7052,train,34,1,0,...,0,0,0,1,0,0,0,0,0,0


In [32]:
#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

In [33]:
data_train= train.drop(columns=['source'])
data_test= test.drop(columns=['source'])

In [34]:
#Export files as modified versions:
data_train.to_csv("train_modified.csv",index=False)
data_test.to_csv("test_modified.csv",index=False)

In [108]:
data_train.to_excel('fe_train.xlsx', index=False)

In [109]:
data_test.to_excel('fe_test.xlsx', index=False)